In [59]:
import pandas as pd
import numpy as np
import math
import lightgbm as lgb
from lightgbm import LGBMRegressor
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


In [2]:
###########################################
######## 1. Data Processing ###############
###########################################

# Read in the core dataset provided by the author
data = pd.read_csv("GKX_20201231.csv",sep=",")

In [61]:
# Examine the dataset
data.head()
data.tail()
# The dimension & column names
print(data.shape)
data.columns

# All columns are either numeric or dummy variables for different categories
print(data.select_dtypes('object').shape[1])

# A function to check for missing values
def missing_val_table(dataframe):
    na_columns = [col for col in dataframe.columns if dataframe[col].isnull().sum() > 0]
    n_miss = dataframe[na_columns].isnull().sum().sort_values(ascending=False)
    ratio = (dataframe[na_columns].isnull().sum() / dataframe.shape[0] * 100).sort_values(ascending=False)
    missing_df = pd.concat([n_miss, np.round(ratio, 2)], axis=1, keys=['n_miss', 'ratio'])
    return missing_df

missing_val_table(data)

(4345508, 101)
Index(['permno', 'DATE', 'mvel1', 'RET', 'prc', 'SHROUT', 'beta', 'betasq',
       'chmom', 'dolvol',
       ...
       'baspread', 'ill', 'maxret', 'retvol', 'std_dolvol', 'std_turn',
       'zerotrade', 'sic2', 'bm', 'bm_ia'],
      dtype='object', length=101)
0


,n_miss,ratio
realestate,3288099,75.67
rd_sale,2928094,67.38
rd_mve,2903841,66.82
secured,2768434,63.71
stdcf,2767345,63.68
stdacc,2767345,63.68
roavol,2307741,53.11
orgcap,2265764,52.14
grltnoa,2260972,52.03
pchsaleinv,2214535,50.96


In [19]:
# We note the data actually start from 1926
# while the paper used data from Mar-1957

#print(data.iloc[0:4,1])
#print(data.iloc[0:4,1])

# We filter out data earlier than March-1957 and later than Dec-2016
df = data[(data['DATE'] > 19570228)].reset_index().iloc[:,1:]
df = df[(df['DATE'] // 10000 < 2017)].reset_index().iloc[:,1:]
print(df.head().iloc[:,0:3])
print(df.tail().iloc[:,0:3])


   permno      DATE      mvel1
0   10006  19570329  86308.500
1   10014  19570329   3903.375
2   10022  19570329   9841.000
3   10030  19570329  51901.000
4   10057  19570329  35250.000
         permno      DATE         mvel1
3760028   93428  20161230  1.250976e+06
3760029   93429  20161230  5.600537e+06
3760030   93433  20161230  2.473625e+04
3760031   93434  20161230  8.573280e+04
3760032   93436  20161230  2.840318e+07


In [58]:
# We know that among the 101 columns, 94 are stock level characteristics
# The sic2 column corresponds to the industry variable
# The other six columns are not used for prediction, according to the appendix of the paper
for i in range(0,len(df.columns)):
    if 10*i < len(df.columns):
        print(data.columns.sort_values()[(10*i):(min(len(df.columns),10*(i+1)))])

        
# Unique number of stocks
#len(np.unique(df['permno']))

Index(['DATE', 'RET', 'SHROUT', 'absacc', 'acc', 'aeavol', 'age', 'agr',
       'baspread', 'beta'],
      dtype='object')
Index(['betasq', 'bm', 'bm_ia', 'cash', 'cashdebt', 'cashpr', 'cfp', 'cfp_ia',
       'chatoia', 'chcsho'],
      dtype='object')
Index(['chempia', 'chinv', 'chmom', 'chpmia', 'chtx', 'cinvest', 'convind',
       'currat', 'depr', 'divi'],
      dtype='object')
Index(['divo', 'dolvol', 'dy', 'ear', 'egr', 'ep', 'gma', 'grcapx', 'grltnoa',
       'herf'],
      dtype='object')
Index(['hire', 'idiovol', 'ill', 'indmom', 'invest', 'lev', 'lgr', 'maxret',
       'mom12m', 'mom1m'],
      dtype='object')
Index(['mom36m', 'mom6m', 'ms', 'mve0', 'mve_ia', 'mvel1', 'nincr', 'operprof',
       'orgcap', 'pchcapx_ia'],
      dtype='object')
Index(['pchcurrat', 'pchdepr', 'pchgm_pchsale', 'pchquick', 'pchsale_pchinvt',
       'pchsale_pchrect', 'pchsale_pchxsga', 'pchsaleinv', 'pctacc', 'permno'],
      dtype='object')
Index(['prc', 'pricedelay', 'ps', 'quick', 'rd', 'rd_mve'

In [ ]:
# Taking the first two digits of SIC2 for industry
df['sic2'] = df['sic2'].apply(np.floor)

# Check whether there are 74 industries as said in the paper
#df.groupby('sic2').count().iloc[:,0:1] 
df['sic2'] = df['sic2'].astype(str)
len(np.unique(df['sic2']))

# Convert the industries to dummies variable
df = pd.get_dummies(df)
df.shape

In [83]:
##################### Import the other dataset mentioned by the author from Welch and Goyal(2008)

# This contains the macroeconomic factors he constructed
# https://sites.google.com/view/agoyal145
# https://drive.google.com/file/d/1ACbhdnIy0VbCWgsnXkjcddiV8HF4feWv/view
df2 = pd.read_csv("PredictorData2020.csv",sep=",")
df2 = df2[df2['yyyymm']>=195603].reset_index().iloc[:,1:]

# Note the index was stored as object
df2.dtypes
# First remove the "," in the index level stored as string
for i in range(0,len(df2['Index'])):
    df2['Index'][i] = df2['Index'][i].replace(',','')
# Then change the index level back to float
df2['Index'] = df2['Index'].apply(float)
df2.dtypes
    
# Compute dividend to price ratio
df2['dp_SnP'] = df2['D12'].apply(math.log) - df2['Index'].apply(math.log)
# Compute earnings to price ratio
df2['ep_SnP'] = df2['E12'].apply(math.log) - df2['Index'].apply(math.log)
# Compute the default yield spread
df2['dfy'] = df2['BAA'] - df2['AAA']
# Term spread = Long term yield - t-bill rate
df2['tms'] = df2['lty'] - df2['tbl']

# Rename the book-to-market ratio since there is another variable of the exact same name in the core dataset
df2 = df2.rename(columns={'b/m': 'bm_DJI'})

# Keep only the eight variables needed
lst = ['dp_SnP','ep_SnP','bm_DJI','ntis','tbl','tms','dfy','svar']
df2 = df2[lst]
df2.head()
df2.tail()

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


,dp_SnP,ep_SnP,bm_DJI,ntis,tbl,tms,dfy,svar
773,-4.080892,-3.569975,0.235975,-0.008504,0.0010,0.0055,0.0102,0.000743
774,-4.045576,-3.533379,0.241482,-0.005698,0.0011,0.0057,0.0105,0.004907
775,-4.020768,-3.519300,0.253146,-0.001895,0.0010,0.0069,0.0109,0.003661
776,-4.126173,-3.635623,0.226352,-0.005262,0.0009,0.0078,0.0100,0.002492
777,-4.165890,-3.686452,0.219195,-0.000094,0.0009,0.0084,0.0090,0.000678


In [ ]:
#df['RET'].describe()

#df2['D12'].apply(math.log)

#df2.head()
#df2.tail()

df2 = df2[lst]
df2.head()
df2.tail()